LIST OF CONTENT - Grouping Data & Aggregating Variables _task

1. preparations
2. gropuping data - identify mean of order_numbers with entire dataset
3. defining max_order and creating max_order column
4. assigning loyalty-flag with loc() function
5. basic statistics of loyalty_flags
6. create a spending_flag
7. create an order_frequency_flag 

# 1. preparations

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path=r'/Users/ru/Desktop/CF/04 Achievement 4_Python'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path,
'02_Data','2.2 Prepared Data', 'ords_prods_merge_4.7.pkl'))

In [4]:
ords_prods_merge.shape

(32435070, 19)

# 2. gropuping data - identify mean of order_numbers with entire dataset

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


As expected the results of the calculation of the mean in the subset and in the whole data set are slightly different while the biggest difference (in department_id 2) is about 3,4 (17,3 - whole dataset vs 20,7 subset) 

# 3. defining max_order and creating max_order column

In [6]:
# defining max_order and creating max_order column
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/ds/6109zp994ddgpkwq7qfc1y3c0000gp/T/ipykernel_17406/2573096944.py:2: FutureWarning: The provided callable <function max at 0x11099a980> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
ords_prods_merge.head(5)

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,last_order (in days),add_to_cart_order,reordered,_merge,price_range_loc,busiest day,busiest days,Most Orders,max_order
0,0.0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,3139998.0,138.0,28.0,6.0,11.0,3.0,5.0,0.0,both,Mid-range product,Regularly busy,Regularly busy,Most Orders,32.0
1,0.0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,1977647.0,138.0,30.0,6.0,17.0,20.0,1.0,1.0,both,Mid-range product,Regularly busy,Regularly busy,Most Orders,32.0
2,0.0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,389851.0,709.0,2.0,0.0,21.0,6.0,20.0,0.0,both,Mid-range product,Busiest day,Busiest day,Most Orders,5.0
3,0.0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,652770.0,764.0,1.0,3.0,13.0,0.0,10.0,0.0,both,Mid-range product,Regularly busy,Least busy,Most Orders,3.0
4,0.0,1,Chocolate Sandwich Cookies,61.0,19.0,5.8,1813452.0,764.0,3.0,4.0,17.0,9.0,11.0,1.0,both,Mid-range product,Least busy,Least busy,Most Orders,3.0


# 4. assigning loyalty-flag with loc() function

In [8]:
# assigning loyalty-categories

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# check results
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15891507
Loyal customer      10294027
New customer         6249525
NaN                       11
Name: count, dtype: int64

# 5. basic statistics of loyalty_flags

In [13]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

compared to the regular and new customers the loyal customers purchase in a lower price-segment. the min and max values are not so interesting since, as mentioned in the text "the minimum orders will always be 1 and the maximum has been capped at 99 by the data engineers at Instacart"

# 6. create a spending_flag

In [14]:
# mean prices and mean_prices column
ords_prods_merge['mean_prices'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/ds/6109zp994ddgpkwq7qfc1y3c0000gp/T/ipykernel_17406/2363129049.py:2: FutureWarning: The provided callable <function mean at 0x11099b380> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_prices'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [15]:
# define and assigne categories for spending flag
ords_prods_merge.loc[ords_prods_merge['mean_prices'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['mean_prices'] >= 10, 'spending_flag'] = 'High spender'

In [16]:
# check output
ords_prods_merge [['user_id', 'max_order', 'loyalty_flag', 'mean_prices','spending_flag']].head(10)

,user_id,max_order,loyalty_flag,mean_prices,spending_flag
0,138.0,32.0,Regular customer,6.935811,Low spender
1,138.0,32.0,Regular customer,6.935811,Low spender
2,709.0,5.0,New customer,7.930208,Low spender
3,764.0,3.0,New customer,4.972414,Low spender
4,764.0,3.0,New customer,4.972414,Low spender
5,777.0,26.0,Regular customer,6.935398,Low spender
6,825.0,9.0,New customer,5.957576,Low spender
7,910.0,12.0,Regular customer,6.680000,Low spender
8,1052.0,20.0,Regular customer,7.162500,Low spender
9,1052.0,20.0,Regular customer,7.162500,Low spender


In [26]:
ords_prods_merge['spending_flag'].value_counts (dropna=False)

spending_flag
Low spender     31800381
High spender      634678
NaN                   11
Name: count, dtype: int64

# 7. create an order_frequency_flag 

according to the median in the “days_since_prior_order” column

In [17]:
# create new column based on frequency median
ords_prods_merge['median_frequency'] = ords_prods_merge.groupby(['user_id'])['last_order (in days)'].transform(np.median)

/var/folders/ds/6109zp994ddgpkwq7qfc1y3c0000gp/T/ipykernel_17406/2912947416.py:2: FutureWarning: The provided callable <function median at 0x110ada020> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_frequency'] = ords_prods_merge.groupby(['user_id'])['last_order (in days)'].transform(np.median)


In [17]:
# create new column based on frequency median
ords_prods_merge['median_frequency'] = ords_prods_merge.groupby(['user_id'])['last_order (in days)'].transform(np.median)

/var/folders/ds/6109zp994ddgpkwq7qfc1y3c0000gp/T/ipykernel_17406/2912947416.py:2: FutureWarning: The provided callable <function median at 0x110ada020> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_frequency'] = ords_prods_merge.groupby(['user_id'])['last_order (in days)'].transform(np.median)


In [18]:
# define and assigne categories for order frequency
ords_prods_merge.loc[ords_prods_merge['median_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['median_frequency'] > 10)& (ords_prods_merge['median_frequency'] <=20),'order_frequency_flag'] = 'Regular customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['median_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [21]:
# check output

In [22]:

ords_prods_merge['order_frequency_flag'].value_counts (dropna=False)

order_frequency_flag
Frequent customer        22816041
Regular customer          6929012
Non-frequent customer     2690006
NaN                            11
Name: count, dtype: int64

In [23]:
ords_prods_merge [['user_id', 'max_order', 'mean_prices','spending_flag', 'loyalty_flag', 'median_frequency','order_frequency_flag']].head(10)

,user_id,max_order,mean_prices,spending_flag,loyalty_flag,median_frequency,order_frequency_flag
0,138.0,32.0,6.935811,Low spender,Regular customer,8.0,Frequent customer
1,138.0,32.0,6.935811,Low spender,Regular customer,8.0,Frequent customer
2,709.0,5.0,7.930208,Low spender,New customer,6.0,Frequent customer
3,764.0,3.0,4.972414,Low spender,New customer,9.0,Frequent customer
4,764.0,3.0,4.972414,Low spender,New customer,9.0,Frequent customer
5,777.0,26.0,6.935398,Low spender,Regular customer,11.0,Regular customer
6,825.0,9.0,5.957576,Low spender,New customer,19.0,Regular customer
7,910.0,12.0,6.680000,Low spender,Regular customer,3.5,Frequent customer
8,1052.0,20.0,7.162500,Low spender,Regular customer,9.5,Frequent customer
9,1052.0,20.0,7.162500,Low spender,Regular customer,9.5,Frequent customer


In [24]:
# export notebook
ords_prods_merge.to_pickle(os.path.join(path, '02_Data','2.2 Prepared Data', 'ords_prods_merge_4.8.pkl'))

In [25]:
ords_prods_merge.shape

(32435070, 25)